In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Libraries

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import sys
import time
import tqdm

import numpy as np
import pandas as pd

import pickle

import string
import io
import re
import nltk
import emoji

import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from wordcloud import WordCloud
from textblob import TextBlob
from nltk.util import ngrams
from nltk import word_tokenize

#from TurkishStemmer import TurkishStemmer
#from zeyrek import MorphAnalyzer
#from zemberek import TurkishSpellChecker
#from zemberek import TurkishSentenceNormalizer
#from zemberek import TurkishSentenceExtractor
#from zemberek import TurkishMorphology
#from zemberek import TurkishTokenizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from mlxtend.plotting import plot_confusion_matrix

from sklearn.metrics import roc_curve, auc, classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

from IPython.display import Markdown, clear_output

def bold(string):
    display(Markdown("**" + string + "**"))

import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

# Load Data

In [ ]:
df = pd.read_excel("/kaggle/input/turkish-tweet-dataset/TurkishTweets.xlsx", sheet_name="Sayfa1")
df.columns = ["text", "label"]
df.head()

In [ ]:
def df_stats(data):
    bold(" SHAPE ".center(50, "#"))
    print("ROWS: {}".format(data.shape[0]))
    print("COLS: {}".format(data.shape[1]))
    bold(" TYPES ".center(50, "#"))
    print(data.dtypes)
    bold(" MISSING VALUES ".center(50, "#"))
    print(data.isnull().sum())
    bold(" DUPLICATED VALUES ".center(50, "#"))
    print("NUMBER OF DUPLICATED VALUES: {}".format(data.duplicated().sum()))
    #bold(" DESCRIBE ".center(50, "#"))
    #print(data.describe().T)
    bold(" MEMORY USAGE ".center(50, "#"))
    buf = io.StringIO()
    data.info(buf=buf)
    info = buf.getvalue().split("\n")[-2].split(":")[1].strip()
    print("Memory Usage: {}".format(info))

In [ ]:
df_stats(df)

In [ ]:
df = df.drop_duplicates()
df = df.dropna()
df = df.reset_index(drop=True)

# Preprocess

In [ ]:
kisaltmalar = {
    "AA":  "Anadolu Ajansı", 
    "AAET":  "Avrupa Atom Enerjisi Topluluğu",
    "AB": "Avrupa Birliği",  
    "ABB": "Avrupa Birliği Bankası",
    "ABGS": "Avrupa Birliği Genel Sekreterliği",
    "ABKF": "Avrupa Bölgesel Kalkınma Fonu",
    "ABÖ": "Afrika Birliği Örgütü",
    "ACÜ": "Acıbadem Üniversitesi", 
    "AÇSAP": "Ana Çocuk Sağlığı ve Aile Planlaması Genel Müdürlüğü", 
    "ADKF": "Abu Dabi Kalkınma Fonu",  
    "ADSL": "Asimetrik sayısal abone hattı (Asymmetric Digital Subscriber Line)", 
    "ADÜ": "Adnan Menderes Üniversitesi", 
    "ADYÜ": "Adıyaman Üniversitesi",
    "AEGM": "Kültür ve Turizm Bakanlığı Araştırma ve Eğitim Genel Müdürlüğü", 
    "AEİBÖ": "Avrupa Ekonomik İş Birliği Örgütü",  
    "AET": "Avrupa Ekonomik Topluluğu",  
    "AEÜ": "Ahi Evran Üniversitesi",
    "AFAD": "Afet ve Acil Durum Yönetimi Başkanlığı", 
    "AFFB": "Avrupa Futbol Federasyonları Birliği",  
    "AFİF": "Akaryakıt Fiyat İstikrar Fonu",
    "AGİK": "Avrupa Güvenlik ve İş Birliği Konferansı", 
    "AGİT": "Avrupa Güvenlik ve İş Birliği Teşkilatı", 
    "AGM": "Ağaçlandırma ve Erozyon Kontrolü Genel Müdürlüğü", 
    "AİBÖ": "Avrupa İş Birliği Örgütü",  
    "AİBÜ": "Abant İzzet Baysal Üniversitesi", 
    "AİÇÜ": "Ağrı İbrahim Çeçen Üniversitesi",
    "AIDS": "Edinilmiş bağışıklık yetersizliği belirtisi",  
    "AİHAD": "Avrupa İnsan Hakları Adalet Divanı",
    "AİHM": "Avrupa İnsan Hakları Mahkemesi",
    "AİHS": "Avrupa İnsan Hakları Sözleşmesi", 
    "AİK": "Afrika İktisat Komisyonu", 
    "AİKB": "Avrupa İmar ve Kalkınma Bankası", 
    "AİPB": "Avrupa İktisadi ve Parasal Birliği",
    "AK": "Avrupa Konseyi", 
    "AKB": "Afrika Kalkınma Bankası", 
    "AKBB": "Afrika Kalkınma Bankaları Birliği", 
    "AKBİL": "Akıllı bilet",
    "AKÇT": "Avrupa Kömür ve Çelik Topluluğu",  
    "AKD": "Akdeniz Üniversitesi" ,
    "AKDTYK": "Atatürk Kültür, Dil ve Tarih Yüksek Kurumu", 
    "AKM": "Atatürk Kültür Merkezi", 
    "AKPM": "Avrupa Konseyi Parlamenterler Meclisi",
    "AKUT": "Arama Kurtarma Teşkilatı",
    "AKÜ": "Afyon Kocatepe Üniversitesi",
    "Alb.": "Albay",
    "ALES": "Akademik Personel ve Lisanüstü Eğitimi Giriş Sınavı", 
    "Alm.":  "Almanca", 
    "AMB": "Amerika Merkez Bankası", 
    "AMK": "Amerikan Merkez Komutanlığı", 
    "ANAM": "Avrupa Nükleer Araştırma Merkezi",  
    "anat.": "Anatomi",
    "ANAÜ": "Anadolu Üniversitesi", 
    "ANT": "Akaryakıt İkmal ve Nato Pol Tesisleri İşletme Başkanlığı",
    "ant.": "Antropoloji", 
    "AO": "Anonim ortaklık", 
    "AOÇ": "Atatürk Orman Çiftliği", 
    "AÖB": "Avrupa Ödemeler Birliği", 
    "AÖF": "Açıköğretim Fakültesi", 
    "AP": "ABD Haber Ajansı", 
    "APA": "Avrupa Para Anlaşması", 
    "APB": "Avrupa Para Birliği", 
    "APE": "Avrupa Para Enstitüsü", 
    "APF": "Arap Para Fonu", 
    "APİİB": "Asya Pasif ik İktisadi İş Birliği",  
    "APÖ": "Avrupa Parlamenterler Örgütü",  
    "APS": "Acele Posta Servisi", 
    "Apt.": "Apartman", 
    "Ar.": "Arapça", 
    "AREL": "İstanbul Arel Üniversitesi", 
    "AR-GE": "Araştırma geliştirme", 
    "ark.": "Arkeoloji", 
    "Arş. Gör.": "Araştırma Görevlisi", 
    "As.": "Asker",
    "As. İz.": "Askerî İnzibat", 
    "ASAL": "Asker Alma Dairesi Başkanlığı", 
    "Asb.": "Astsubay", 
    "ASELSAN": "Askerî Elektronik Sanayi", 
    "ASF": "Avrupa Sosyal Fonu",
    "ASİGM": "Adli Sicil ve İstatistik Genel Müdürlüğü", 
    "ASK": "Avrupa Standartlar Komitesi", 
    "ASKİ": "Ankara Su ve Kanalizasyon İdaresi", 
    "ASO": "Ankara Sanayi Odası", 
    "AST": "Ankara Sanat Tiyatrosu", 
    "ASTB": "Amerika Serbest Ticaret Bölgesi",
    "astr.": "Astronomi", 
    "astrol.": "Astroloji", 
    "ASÜ": "Aksaray Üniversitesi", 
    "AŞ": "Anonim şirket", 
    "AŞTİ": "Ankara Şehirler Arası Terminal İşletmesi", 
    "AT": "Avrupa Topluluğu",
    "ATAD": "Avrupa Toplulukları Adalet Divanı", 
    "ATAM": "Atatürk Araştırma Merkezi", 
    "ATATÜRK ÜNİ": "Atatürk Üniversitesi",
    "ATB": "Asya Takas Birliği", 
    "Atğm.": "Asteğmen", 
    "ATK": "Adli Tıp Kurumu", 
    "atm.": "Atmosfer", 
    "ATO": "Ankara Ticaret Odası", 
    "ATÜ": "Atılım Üniversitesi", 
    "ATYGF": "Avrupa Tarımsal Yönlendirme ve Garanti Fonu",  
    "AUA": "Avrupa Uzay Ajansı", 
    "AÜ": "Ankara Üniversitesi",
    "Av.": "Avukat", 
    "AVM": "Alışveriş merkezi", 
    "AYB": "Avrupa Yayın Birliği", 
    "AYİM": "Askerî Yüksek İdare Mahkemesi", 
    "AYM": "Anayasa Mahkemesi", 
    "BADET": "Batı Afrika Devletleri Ekonomik Topluluğu", 
    "BAE": "Birleşik Arap Emirlikleri", 
    "bağ.": "Bağlaç", 
    "BAĞ-KUR": "Esnaf ve Sanatkârlar ve Diğer Bağımsız Çalışanlar Sosyal Sigortalar Kurumu", 
    "BAKB": "Batı Afrika Kalkınma Bankası",
    "BASAGM": "Başbakanlık Aile ve Sosyal Araştırmalar Genel Müdürlüğü", 
    "BATO": "Batı Afrika Takas Odası", 
    "BAU": "Bahçeşehir Üniversitesi", 
    "BAÜN": "Balıkesir Üniversitesi", 
    "BAY ÜNİ": "Bayburt Üniversitesi", 
    "BBC": "İngiliz Radyo Televizyon Kurumu",  
    "BCG": "Verem aşısı", 
    "Bçvş.": "Başçavuş", 
    "BD": "Bilim dalı", 
    "BDDK": "Bankacılık Düzenleme ve Denetleme Kurulu", 
    "BDT": "Bağımsız Devletler Topluluğu",  
    "BE": "Büyükelçi", 
    "BEÜ": "Bitlis Eren Üniversitesi", 
    "BİB": "Bayındırlık ve İskân Bakanlığı", 
    "BİK": "Basın İlan Kurumu",  
    "bit. b.": "Bitki bilimi", 
    "biy.": "Biyoloji", 
    "bk.": "Bakınız", 
    "bl.": "Bilişim", 
    "Bl.": "Bölük", 
    "BM": "Birleşmiş Milletler", 
    "BMAEK": "Birleşmiş Milletler Avrupa Ekonomik Komisyonu", 
    "BMÇYT": "Birleşmiş Milletler Çocuklara Yardım Teşkilatı", 
    "BMEBKT": "Birleşmiş Milletler Eğitim, Bilim ve Kültür Teşkilatı", 
    "BMSKÖ": "Birleşmiş Milletler Sınai Kalkınma Örgütü", 
    "BMT": "Birleşmiş Milletler Teşkilatı",  
    "BMTKK": "Birleşmiş Milletler Ticaret ve Kalkınma Konferansı",
    "BMYRİ": "Birleşmiş Milletler Yardım ve Rehabilitasyon İdaresi",  
    "Bn.": "Bayan", 
    "Bnb.": "Binbaşı", 
    "BOREN": "Ulusal Bor Araştırma Enstitüsü ",
    "BOTAŞ": "Boru Hatları ile Petrol Taşıma Anonim Şirketi", 
    "Böl.": "Bölüm", 
    "BRT": "Bartın Üniversitesi", 
    "bs.": "Baskı",
    "Bşk.": "Başkan", 
    "BTF": "Başbakanlık Tanıtma Fonu Kurulu", 
    "BTK": "Bilgi Teknolojileri ve İletişim Kurumu", 
    "BTÜ": "Bursa Teknik Üniversitesi",
    "Bul.": "Bulvar", 
    "Bulg.": "Bulgarca", 
    "BÜ": "Boğaziçi Üniversitesi",
    "BÜMKO": "Bütçe ve Mali Kontrol Genel Müdürlüğü",
    "BVÜ": "Bezmiâlem Vakıf Üniversitesi", 
    "BYEGM": "Basın-Yayın ve Enformasyon Genel Müdürlüğü", 
    "Cad.": "Cadde",
    "CBÜ": "Celâl Bayar Üniversitesi", 
    "CC": "Konsolosluk birimi",  
    "CGS": "Uluslararası fizik birimleri sistemi", 
    "CIF": "Maliyet-sigorta-navlun",
    "CİGM": "Ceza İşleri Genel Müdürlüğü", 
    "CIP": "Taşıma ve sigorta bedeli ödenmiş olarak teslim",
    "cm": "Santimetre", 
    "CMK": "Ceza Muhakemesi Kanunu", 
    "CMUK": "Ceza Muhakemeleri Usulü Kanunu", 
    "coğ.": "Coğrafya", 
    "cos": "Kosinüs",
    "CSO": "Cumhurbaşkanlığı Senfoni Orkestrası", 
    "CTE": "Ceza ve Tevkifevleri Genel Müdürlüğü", 
    "Cum. Bşk.": "Cumhurbaşkanı", 
    "CÜ": "Cumhuriyet Üniversitesi", 
    "ÇAĞ ÜNİ": "Çağ Üniversitesi", 
    "ÇASGEM": "Çalışma ve Sosyal Güvenlik Eğitim ve Araştırma Merkezi", 
    "ÇAYKUR": "Çay İşletmeleri Genel Müdürlüğü", 
    "ÇED": "Çevresel Etki Değerlendirmesi ve Planlama Genel Müdürlüğü",
    "çev.": "Çeviren", 
    "ÇGM": "Çalışma ve Sosyal Güvenlik Bakanlığı Çalışma Genel Müdürlüğü", 
    "ÇNÜ": "Çankırı Karatekin Üniversitesi", 
    "ÇOMÜ": "Çanakkale Onsekiz Mart Üniversitesi", 
    "ÇS": "Çimento Sanayisi", 
    "ÇSGB": "Çalışma ve Sosyal Güvenlik Bakanlığı", 
    "ÇUKOBİRLİK": "Çukurova Pamuk Tarım Satış Kooperatifleri Birliği", 
    "ÇÜ": "Çankaya Üniversitesi", 
    "Çvş.": "Çavuş", 
    "ÇYGM": "Çevre Yönetimi Genel Müdürlüğü", 
    "DAGM": "Devlet Arşivleri Genel Müdürlüğü", 
    "DAKB": "Doğu Afrika Kalkınma Bankası",
    "dal": "Dekalitre", 
    "dam": "Dekametre", 
    "DAP": "Doğu Anadolu Projesi", 
    "DAT": "Doğu Afrika Topluluğu", 
    "DB": "Dünya Bankası", 
    "db.": "Dil bilgisi", 
    "DBAFİT": "Dünya Bankalar Arası Finansal İletişim Topluluğu", 
    "DDB": "Dernekler Dairesi Başkanlığı",
    "DDY": "Devlet Deniz Yolları", 
    "DESİYAB": "Devlet Sanayi ve İşçi Yatırım Bankası", 
    "DEÜ": "Dokuz Eylül Üniversitesi",
    "DFİF": "Destekleme ve Fiyat İstikrar Fonu", 
    "DFMHÖ": "Dünya Fikrî Mülkiyet Hakları Örgütü",
    "dg": "Desigram",
    "DGK": "Dünya Gıda Konseyi", 
    "DGM": "Devlet Güvenlik Mahkemesi", 
    "DGÖ": "Dünya Gümrük Örgütü", 
    "DGS": "Dikey Geçiş Sınavı", 
    "DHMİ": "Devlet Hava Meydanları İşletmesi",
    "DİB": "Diyanet İşleri Başkanlığı", 
    "DİE": "Devlet İstatistik Enstitüsü", 
    "dil b.": "Dil bilimi", 
    "DİYİH": "Çalışma ve Sosyal Güvenlik Bakanlığı Dış İlişkiler ve Yurt Dışı İşçi Hizmetleri Genel Müdürlüğü", 
    "dk.": "Dakika", 
    "DKK": "Deniz Kuvvetleri Komutanlığı",
    "DKMP": "Doğa Koruma ve Millî Parklar Genel Müdürlüğü", 
    "dl": "Desilitre", 
    "DLH": "Demiryollar Limanlar ve Hava Meydanları İnşaatı Genel Müdürlüğü", 
    "dm": "Desimetre", 
    "DMİ": "Devlet Meteoroloji İşleri Genel Müdürlüğü", 
    "DMO": "Devlet Malzeme Ofisi", 
    "DNA": "Deoksirübo nükleik asit", 
    "DOBGM": "Devlet Opera ve Balesi Genel Müdürlüğü", 
    "Doç.": "Doçent", 
    "doğ.": "Doğum tarihi", 
    "DOÜ": "Doğuş Üniversitesi",
    "DÖGM": "Din Öğretimi Genel Müdürlüğü", 
    "DÖSİM": "Kültür ve Turizm Bakanlığı Döner Sermaye İşletmesi Merkez Müdürlüğü",
    "DPB": "Devlet Planlama Başkanlığı",
    "DPT": "Devlet Planlama Teşkilatı Müsteşarlığı", 
    "DPÜ": "Dumlupınar Üniversitesi", 
    "Dr.": "Doktor", 
    "drl.": "Derleyen", 
    "DSİ": "Devlet Su İşleri Genel Müdürlüğü", 
    "DSO": "Devlet Sanayi Ofisi", 
    "DSÖ": "Dünya Sağlık Örgütü", 
    "DT": "Devlet Tiyatroları Genel Müdürlüğü", 
    "DTCF": "Dil ve Tarih-Coğrafya Fakültesi", 
    "DTM": "Dış Ticaret Müsteşarlığı",
    "DTÖ": "Dünya Ticaret Örgütü", 
    "DÜ": "Dicle Üniversitesi", 
    "Dz. Kuv. K.": "Deniz Kuvvetleri Komutanı", 
    "Dz. Kuv.": "Deniz Kuvvetleri", 
    "dzl.": "Düzenleyen", 
    "EAT": "Eski Anadolu Türkçesi", 
    "EBK": "Et ve Balık Kurumu Genel Müdürlüğü", 
    "Ecz.": "Eczacı", 
    "ed.": "Editör", 
    "EEG": "Beyin akım grafiği", 
    "EFT": "Bankalar arası para aktarımı",
    "EGM": "Emniyet Genel Müdürlüğü", 
    "EGO": "Ankara Belediyesi Elektrik, Gaz, Otobüs İşletmeleri", 
    "EĞİTEK": "Eğitim Teknolojileri Genel Müdürlüğü", 
    "EİBKT": "Ekonomik İş Birliği ve Kalkınma Teşkilatı", 
    "EİBÖ": "Ekonomik İş Birliği Örgütü",
    "EİEİ": "Elektrik İşleri Etüt İdaresi Genel Müdürlüğü", 
    "EKG": "Kalp akım grafiği", 
    "ekon.": "Ekonomi", 
    "EMG": "Kasakım grafiği",
    "EMK": "Elektromotor kuvvet", 
    "Ens.": "Enstitü", 
    "EPDK": "Enerji Piyasası Düzenleme Kurulu", 
    "ERDEMİR": "Ereğli Demir Çelik Fabrikaları", 
    "Erm.": "Ermenice", 
    "ERÜ": "Erciyes Üniversitesi", 
    "ESGM": "Esnaf ve Sanatkârlar Genel Müdürlüğü", 
    "ESHOT": "İzmir Belediyesi Elektrik, Su, Hava Gazı, Otobüs, Tramvay İşletmeleri", 
    "ESOGÜ": "Eskişehir Osmangazi Üniversitesi", 
    "ET": "Eski Türkçe",
    "ETKB": "Enerji ve Tabii Kaynaklar Bakanlığı", 
    "ETOGM": "Erkek Teknik Öğretim Genel Müdürlüğü",
    "EÜ": "Ege Üniversitesi", 
    "EÜAŞ": "Elektrik Üretim Anonim Şirketi", 
    "EXİMBANK": "Türkiye İhracat ve Kredi Bankası", 
    "FAB": "Federal Araştırma Bürosu", 
    "Fak.": "Fakülte", 
    "Fak-Fuk": "Fon Sosyal Yardımlaşma ve Dayanışmayı Teşvik Fonu", 
    "Far.": "Farsça", 
    "FBA": "Fransız Basın Ajansı",  
    "fel.": "Felsefe", 
    "fil.": "Filoloji", 
    "FIR": "Uçuş hattı düzenlemesi",
    "FİSKOBİRLİK": "Fındık Tarım Satış Kooperatifleri Birliği", 
    "fiz.": "Fizik", 
    "fizy.": "Fizyoloji", 
    "FKB": "Fizik, kimya, biyoloji", 
    "FM": "Frekans modülasyonu",
    "IÜ": "Işık Üniversitesi",
    "Fr.": "Fransızca", 
    "FSMVÜ": "Fatih Sultan Mehmet Vakıf Üniversitesi", 
    "FÜ": "Fırat Üniversitesi",
    "g": "Gram", 
    "GAMHK": "Güney Avrupa Müttefik Hava Kuvvetleri",
    "GAP": "Güneydoğu Anadolu Projesi", 
    "GAPBKİ": "GAP Bölge Kalkınma İdaresi Başkanlığı", 
    "GATA": "Gülhane Askerî Tıp Akademisi", 
    "GAUT": "Güneydoğu Asya Uluslar Topluluğu",
    "GAZÜ": "Gaziantep Üniversitesi",
    "GB": "Gigabyte",
    "GBT": "Genel Bilgi Taraması", 
    "GDAOP": "Güney ve Doğu Afrika Ortak Pazarı",
    "Gen.": "General",
    "geom.": "Geometri", 
    "GİB": "Gelir İdaresi Başkanlığı", 
    "GİGM": "Başbakanlık Güvenlik İşleri Genel Müdürlüğü", 
    "gn.": "Genel",
    "Gnkur.": "Genelkurmay",
    "GOÜ": "Gaziosmanpaşa Üniversitesi", 
    "Gön.": "Gönderen", 
    "gr.": "Gramer", 
    "Grt": "Groston", 
    "GSGM": "Gençlik ve Spor Genel Müdürlüğü", 
    "GSM": "Mobil cep telefonu", 
    "GSMH": "Gayrisafi millî hasıla", 
    "GSÜ": "Galatasaray Üniversitesi", 
    "GSYH": "Gayrisafi yurt içi hasıla", 
    "GTÖ": "Gıda ve Tarım Örgütü",
    "GÜ": "Gazi Üniversitesi", 
    "GYD": "Görüntülü yoğun disk", 
    "GYTE": "Gebze Yüksek Teknoloji Enstitüsü", 
    "GZK": "Gazikent Üniversitesi",
    "HA": "Harp Akademisi",
    "HABITAT": "Birleşmiş Milletler İnsan Yerleşimleri Konferansı", 
    "HACCP": "Kritik kontrol noktalarında tehlike analizi", 
    "HAVAŞ": "Hava Alanları ve Yer Hizmetleri Genel Müdürlüğü", 
    "hay. b.": "Hayvan bilimi", 
    "haz.": "Hazırlayan", 
    "HDD": "Ana bellek sürücüsü",   
    "hek.": "Hekimlik", 
    "HGK": "Harita Genel Komutanlığı", 
    "HGM": "Haberleşme Genel Müdürlüğü", 
    "HİTÜ": "Hitit Üniversitesi", 
    "HKK": "Hava Kuvvetleri Komutanlığı", 
    "hl": "Hektolitre", 
    "hlk.": "Halk ağzı", 
    "HM": "Hazine Müsteşarlığı", 
    "hm": "Hektometre", 
    "HMUK": "Hukuk Muhakemeleri Usulü Kanunu", 
    "HO": "Harp Okulu",
    "HRÜ": "Harran Üniversitesi",
    "Hs. Uzm.": "Hesap Uzmanı", 
    "HSSGM": "Sağlık Bakanlığı Hudut ve Sahiller Sağlık Genel Müdürlüğü", 
    "HUK": "Maliye Bakanlığı Hesap Uzmanları Kurulu Başkanlığı", 
    "huk.": "Hukuk", 
    "HÜ": "Hacettepe Üniversitesi", 
    "Hv. Kuv. K.": "Hava Kuvvetleri Komutanı", 
    "Hv. Kuv.": "Hava Kuvvetleri", 
    "Hz. öz.": "Hizmete özel", 
    "Hz.": "Hazreti",  
    "IĞDÜ": "Iğdır Üniversitesi", 
    "İAÜ": "İstanbul Aydın Üniversitesi", 
    "İbr.": "İbranice", 
    "İBÜ": "İstanbul Bilim Üniversitesi", 
    "İBÜN": "İstanbul Bilgi Üniversitesi", 
    "İDBÜ": "İhsan Doğramacı Bilkent Üniversitesi",
    "İDT": "İktisadi Devlet Teşekkülleri", 
    "İEGM": "İlaç ve Eczacılık Genel Müdürlüğü", 
    "İETT": "İstanbul Elektrik, Tünel, Tramvay İşletmesi", 
    "İEÜ": "İzmir Ekonomi Üniversitesi",
    "İGB": "Başbakanlık İdareyi Geliştirme Başkanlığı", 
    "İGEME": "İhracatı Geliştirme Etüd Merkezi", 
    "İGM": "İlköğretim Genel Müdürlüğü", 
    "İHB": "İnsan Hakları Başkanlığı", 
    "İKB": "İslam Kalkınma Bankası", 
    "İKBÜ": "İstanbul Kemerburgaz Üniversitesi", 
    "İKÖ": "İslam Konferansı Örgütü",  
    "İKÜ": "İstanbul Kültür Üniversitesi", 
    "İKV": "İktisadi Kalkınma Vakfı", 
    "İLBANK": "İller Bankası Genel Müdürlüğü", 
    "İMKB": "İstanbul Menkul Kıymetler Borsası", 
    "İng.": "İngilizce", 
    "INTERPOL": "Uluslararası Cinayet Polisi Komisyonu",  
    "İO": "İlköğretim okulu",
    "is.": "İsim",
    "ISBN": "Uluslararası standart kitap numarası",  
    "İSDEMİR": "İskenderun Demir Çelik Fabrikaları", 
    "İSEDAK": "İslam Konferansı Örgütü Ekonomik ve Ticari İş Birliği Daimî Komitesi", 
    "İSG": "İş Sağlığı ve Güvenliği Genel Müdürlüğü", 
    "İSKİ": "İstanbul Su ve Kanalizasyon İdaresi", 
    "İSKYF": "İktisadi ve Sosyal Kalkınma Yatırım Fonları",  
    "ISMN": "Uluslararası standart müzik numarası", 
    "İSO": "İstanbul Sanayi Odası", 
    "İsp.": "İspanyolca", 
    "ISSN": "Uluslararası standart dizi numarası", 
    "İŞKUR": "Türkiye İş Kurumu",
    "işl.": "İşleyen", 
    "İŞOT": "İstanbul Şehirler Arası Otobüs Terminali", 
    "İt.": "İtalyanca", 
    "İTB": "İstanbul Ticaret Borsası", 
    "İTG": "İç Ticaret Genel Müdürlüğü", 
    "İTİCÜ": "İstanbul Ticaret Üniversitesi", 
    "ITO": "Birleşmiş Milletler Uluslararası Ticaret Birliği",
    "İTO": "İstanbul Ticaret Odası", 
    "İTÜ": "İstanbul Teknik Üniversitesi", 
    "İÜ": "İstanbul Üniversitesi", 
    "İYTE": "İzmir Yüksek Teknoloji Enstitüsü", 
    "İZTB": "İzmir Ticaret Borsası", 
    "İZTO": "İzmir Ticaret Odası", 
    "Jap.": "Japonca", 
    "jeol.": "Jeoloji", 
    "JGK": "Jandarma Genel Komutanlığı", 
    "JHY": "Japon Hava Yolları", 
    "kal": "Kalori", 
    "KARDEMİR": "Karabük Demir Çelik Fabrikaları", 
    "KASTB": "Kuzey Amerika Serbest Ticaret Bölgesi",
    "KAÜ": "Kafkas Üniversitesi", 
    "KBB": "Kulak, Burun, Boğaz", 
    "KBÜ": "Karabük Üniversitesi", 
    "KDV": "Katma değer vergisi", 
    "KED": "Kablosuz Erişim Düzeneği", 
    "KEGM": "Kıyı Emniyeti Genel Müdürlüğü", 
    "KEİ": "Karadeniz Ekonomik İş Birliği",
    "KEYK": "Karşılıklı Ekonomik Yardım Konseyi",  
    "kg": "Kilogram", 
    "KGB": "Sovyet Gizli Haber Alma Teşkilatı", 
    "KGM": "Adalet Bakanlığı Kanunlar Genel Müdürlüğü", 
    "KHK": "Kanun hükmünde kararname",
    "KHÜ": "Kadir Has Üniversitesi", 
    "KHz": "Kilohertz", 
    "KİBİB": "Kalkınma İçin Bölgesel İş Birliği",  
    "KİK": "Kamu İhale Kurumu",  
    "kim.": "Kimya", 
    "KİT": "Kamu İktisadi Teşekkülü", 
    "KİYÜ": "Kilis 7 Aralık Üniversitesi", 
    "KKB": "Karayip Kalkınma Bankası",  
    "KKGM": "Tarım ve Köyişleri Bakanlığı Koruma ve Kontrol Genel Müdürlüğü", 
    "KKK": "Kara Kuvvetleri Komutanlığı", 
    "KKTC": "Kuzey Kıbrıs Türk Cumhuriyeti",
    "KLÜ": "Kırklareli Üniversitesi", 
    "km": "Kilometre", 
    "KMS": "Kurumlar İçi Merkezî Eleme Sınavı", 
    "KMÜ": "Karamanoğlu Mehmetbey Üniversitesi", 
    "KOBİ": "Küçük ve Orta Boy İşletme", 
    "KOİ": "Kamu Ortaklığı İdaresi",
    "koor.": "Koordinasyon", 
    "Kor.": "Kolordu", 
    "Kora.": "Koramiral", 
    "Korg.": "Korgeneral", 
    "KOSGEB": "Küçük ve Orta Ölçekli İşletmeleri Geliştirme ve Destekleme İdaresi Başkanlığı", 
    "KOÜ": "Kocaeli Üniversitesi", 
    "KPDS": "Kamu Personeli Yabancı Dil Bilgisi Seviye Tespit Sınavı", 
    "KPSS": "Kamu Personel Seçme Sınavı", 
    "kr.": "Kuruş",
    "krş.": "Karşılaştırınız", 
    "KSDB": "Kanserle Savaş Dairesi Başkanlığı", 
    "KSGM": "Kadının Statüsü Genel Müdürlüğü", 
    "KSÜ": "Kahramanmaraş Sütçü İmam Üniversitesi", 
    "KTKB": "Karadeniz Ticaret ve Kalkınma Bankası", 
    "KTO": "Karatay Üniversitesi", 
    "KTOP": "Karayip Topluluğu ve Ortak Pazarı", 
    "KTÖGM": "Kız Teknik Öğretim Genel Müdürlüğü",
    "KTÜ": "Karadeniz Teknik Üniversitesi", 
    "KUGM": "Kara Ulaştırma Genel Müdürlüğü",
    "Kur. Bşk.":  "Kurmay Başkanı", 
    "Kur.": "Kurmay", 
    "KÜ": "Kastamonu Üniversitesi",
    "KVMGM": "Kültür Varlıkları ve Müzeler Genel Müdürlüğü", 
    "KYGM": "Kütüphaneler ve Yayımlar Genel Müdürlüğü", 
    "l": "Litre",
    "LABB": "Latin Amerika Bütünleşme Birliği",  
    "LAKEK": "Latin Amerika ve Karayipler Ekonomik Komisyonu", 
    "LASTB": "Latin Amerika Serbest Ticaret Bölgesi",  
    "Lat.": "Latince", 
    "LCV": "Lütfen cevap veriniz", 
    "LES": "Lisansüstü Eğitim Giriş Sınavı", 
    "LP": "Uzunçalar",   
    "LPG": "Sıvılaştırılmış Petrol Gazı",  
    "Ltd.": "Limitet", 
    "LYS": "Lisans Yerleştirme Sınavı", 
    "m": "Metre", 
    "M": "Miladi", 
    "Mac.": "Macarca", 
    "Mah.": "Mahalle", 
    "MAKÜ": "Mehmet Akif Ersoy Üniversitesi", 
    "man.": "Mantık", 
    "MAÖ": "Merkezî Antlaşma Örgütü", 
    "MASAK": "Mali Suçları Araştırma Kurulu Başkanlığı", 
    "mat.": "Matematik", 
    "MAÜ": "Mardin Artuklu Üniversitesi", 
    "MB": "Megabyte",  
    "Md.": "Müdür", 
    "MEB": "Millî Eğitim Bakanlığı",  
    "mec.": "Mecaz", 
    "MEÜ": "Mersin Üniversitesi", 
    "MGK": "Millî Güvenlik Kurulu Genel Sekreterliği", 
    "MGM": "İçişleri Bakanlığı Mahalli İdareler Genel Müdürlüğü", 
    "MĞÜ": "Muğla Üniversitesi",
    "MHAT": "Merkezî Haber Alma Teşkilatı",  
    "MHz": "Megahertz", 
    "MİGEM": "Maden İşleri Genel Müdürlüğü", 
    "MİLE": "Millî Emlak Genel Müdürlüğü",
    "mim.": "Mimarlık", 
    "min.": "Mineraloji", 
    "MİT": "Millî İstihbarat Teşkilatı Müsteşarlığı", 
    "MK": "Medeni Kanun",  
    "MKB": "Millî Kütüphane Başkanlığı", 
    "MKE": "Makine ve Kimya Endüstrisi Kurumu",  
    "MKS": "Metre-Kilogram-Saat", 
    "MKÜ": "Mustafa Kemal Üniversitesi", 
    "MKYK": "Merkez Karar Yönetim Kurulu",
    "mm": "Milimetre", 
    "MOBESE": "Bölgesel güvenlik sistemi", 
    "MOSSAD": "İsrail Gizli Haber Alma Örgütü", 
    "MÖ": "Milattan önce", 
    "MPİ": "Millî Piyango İdaresi Genel Müdürlüğü", 
    "MPM": "Millî Prodüktivite Merkezi", 
    "MR": "Manyetik rezonans görüntüleme", 
    "MS": "Milattan sonra", 
    "MSB": "Millî Savunma Bakanlığı", 
    "MSGSÜ": "Mimar Sinan Güzel Sanatlar Üniversitesi", 
    "MTA": "Maden Tetkik ve Arama Genel Müdürlüğü", 
    "MÜ": "Marmara Üniversitesi", 
    "Müh.": "Mühendis", 
    "Mür.": "Müracaat", 
    "müz.": "Müzik", 
    "MYK": "Mesleki Yeterlilik Kurumu", 
    "NASA": "Amerikan Ulusal Havacılık ve Uzay Araştırmaları Merkezi",  
    "NATO": "Kuzey Atlantik Antlaşması Teşkilatı", 
    "NEÜ": "Nevşehir Üniversitesi", 
    "NĞÜ": "Niğde Üniversitesi", 
    "NKÜ": "Namık Kemal Üniversitesi", 
    "No.": "Numara", 
    "NOTAM": "Havacılar ve denizciler için uyarı bildirimi", 
    "Nö.": "Nöbetçi", 
    "Nö. Sb.": "Nöbetçi subayı", 
    "NVİGM": "Nüfus ve Vatandaşlık İşleri Genel Müdürlüğü", 
    "OAİB": "Orta Anadolu İhracatçı Birlikleri Genel Sekreterliği",
    "OAOP": "Orta Amerika Ortak Pazarı",  
    "ODTÜ": "Orta Doğu Teknik Üniversitesi",
    "ODÜ": "Ordu Üniversitesi",
    "OGM": "Orman Genel Müdürlüğü", 
    "OKÖNEGM": "Okul Öncesi Eğitimi Genel Müdürlüğü", 
    "Okt.": "Okutman", 
    "OKÜ": "Osmaniye Korkut Ata Üniversitesi", 
    "OMÜ": "Ondokuz Mayıs Üniversitesi",
    "Onb.": "Onbaşı", 
    "Opr.": "Operatör", 
    "Or.": "Ordu", 
    "Ora.": "Oramiral", 
    "Ord.": "Ordinaryüs", 
    "Org.": "Orgeneral",
    "ORKÖY": "Orman-Köy İlişkileri Genel Müdürlüğü", 
    "Ort.": "Ortaklık", 
    "OSB": "Organize Sanayi Bölgesi", 
    "Osm. T.": "Osmanlı Türkçesi", 
    "OYAK": "Ordu Yardımlaşma Kurumu", 
    "OYEGM": "Öğretmen Yetiştirme ve Eğitimi Genel Müdürlüğü", 
    "ÖÇKKB": "Özel Çevre Koruma Kurumu Başkanlığı", 
    "ÖİB": "Özelleştirme İdaresi Başkanlığı", 
    "öl.": "Ölüm tarihi", 
    "ÖÖKGM": "Özel Öğretim Kurumları Genel Müdürlüğü", 
    "ör.": "Örnek", 
    "ÖRGM": "Özel Eğitim Rehberlik ve Danışma Hizmetleri Genel Müdürlüğü", 
    "ÖSGM": "Sanayi ve Ticaret Bakanlığı Ölçüler ve Standartlar Genel Müdürlüğü", 
    "ÖSS": "Öğrenci Seçme Sınavı", 
    "ÖSYM": "Ölçme, Seçme ve Yerleştirme Merkezi", 
    "ÖTV": "Özel tüketim vergisi", 
    "ÖYK": "Özelleştirme Yüksek Kurulu", 
    "ÖYS": "Öğrenci Yerleştirme Sınavı",
    "öz.": "Özel", 
    "ÖZİ": "Özürlüler İdaresi Başkanlığı",
    "ÖZÜ": "Özyeğin Üniversitesi", 
    "PAÜ": "Pamukkale Üniversitesi", 
    "ped.": "Pedagoji", 
    "PETKİM": "Petrokimya Endüstrisi Genel Müdürlüğü", 
    "PİEAÜÖ": "Petrol İhraç Eden Arap Ülkeleri Örgütü",  
    "PİEÜT": "Petrol İhraç Eden Ülkeler Teşkilatı",
    "PİGM": "Petrol İşleri Genel Müdürlüğü", 
    "PK": "Posta kutusu", 
    "PO": "Petrol Ofisi", 
    "POAŞ": "Petrol Ofisi Anonim Şirketi", 
    "Port.": "Portekizce", 
    "Prof.": "Profesör", 
    "psikol.": "Psikoloji", 
    "PTT": "Posta ve Telgraf Teşkilatı Genel Müdürlüğü",
    "RAM": "Bilgisayarda okunabilir ve yazılabilir bellek", 
    "RC": "Kızılhaç",
    "RK": "Rekabet Kurumu", 
    "RNA": "Ribo nükleik asit", 
    "ROM": "Bilgisayarda yalnızca okunabilir bellek", 
    "RSHM": "Refik Saydam Hıfzıssıhha Merkezi Başkanlığı",
    "RTÜK": "Radyo ve Televizyon Üst Kurulu", 
    "Rum.": "Rumca", 
    "Rus.": "Rusça", 
    "RÜ": "Rize Üniversitesi",
    "s.": "Sayfa", 
    "sa.": "Saat", 
    "SAS": "Su Altı Savunma", 
    "SAT": "Su Altı Taarruz",
    "SATEM": "Savunma Sanayii ve Teknoloji Eğitim Merkezi", 
    "SAÜ": "Sakarya Üniversitesi", 
    "Sb.": "Subay", 
    "SB": "Sağlık Bakanlığı", 
    "SBF": "Siyasal Bilgiler Fakültesi", 
    "SBS": "Seviye Belirleme Sınavı", 
    "SDÜ": "Süleyman Demirel Üniversitesi", 
    "SEFD Bşk.": "Millî Savunma Bakanlığı Seferberlik Dairesi Başkanlığı", 
    "SEK": "Süt Endüstrisi Kurumu Sanayi ve Ticaret Anonim Şirketi", 
    "SEKA": "Selüloz ve Kâğıt Sanayii Kurumu",
    "sf.": "Sıfat", 
    "SG": "Sahil Güvenlik", 
    "SGK": "Sosyal Güvenlik Kurumu",  
    "SGM": "Sanayi ve Ticaret Bakanlığı Sanayi Genel Müdürlüğü", 
    "SHÇEK": "Sosyal Hizmetler ve Çocuk Esirgeme Kurumu", 
    "SHGM": "Sivil Havacılık Genel Müdürlüğü", 
    "SHÖ": "Sivil Havacılık Örgütü", 
    "SİÜ": "Siirt Üniversitesi", 
    "SKT": "Son kullanma tarihi", 
    "Sl.": "Slavca", 
    "Sn.": "Sayın", 
    "sn.": "Saniye", 
    "snt.": "Santral", 
    "SNÜ": "Sinop Üniversitesi", 
    "Sok.": "Sokak", 
    "SOS": "Uluslararası yardım isteme uyarısı",
    "sos.": "Sosyoloji", 
    "sp.": "Spor", 
    "SPK": "Sermaye Piyasası Kurulu",
    "Srp.": "Sırpça", 
    "SSCB": "Sovyet Sosyalist Cumhuriyetleri Birliği",  
    "SSDB": "Sağlık Bakanlığı Sıtma Savaş Dairesi Başkanlığı", 
    "SSK": "Sosyal Sigortalar Kurumu", 
    "SSM": "Savunma Sanayi Müsteşarlığı", 
    "STB": "Sanayi ve Ticaret Bakanlığı", 
    "STK": "Sivil Toplum Kuruluşu", 
    "STÖ": "Sivil Toplum Örgütü", 
    "SUGES": "Sudan gelen sağlık", 
    "SÜ": "Sabancı Üniversitesi",  
    "SYD": "Sayısal yoğun disk", 
    "SYDGM": "Sosyal Yardımlaşma ve Dayanışma Genel Müdürlüğü", 
    "Şb.": "Şube", 
    "ŞÜ": "Şırnak Üniversitesi",
    "t": "Ton", 
    "T.C.": "Türkiye Cumhuriyeti",
    "TAA": "Türkiye Adalet Akademisi", 
    "TAEK": "Türkiye Atom Enerjisi Kurumu", 
    "TAGEM": "Tarımsal Araştırmalar Genel Müdürlüğü", 
    "tar.": "Tarih", 
    "TARİŞ": "Tarım Satış Kooperatifleri Birliği", 
    "TASS": "Sovyetler Birliği Telgraf Ajansı", 
    "TAŞ": "Türk Anonim Şirketi",
    "TAU": "Teknik Araştırma ve Uygulama Genel Müdürlüğü", 
    "Tb.": "Tabur", 
    "TBB": "Türkiye Bankalar Birliği", 
    "TBMM": "Türkiye Büyük Millet Meclisi", 
    "TCDD": "Türkiye Cumhuriyeti Devlet Demiryolları", 
    "TCK": "Türkiye Cumhuriyeti Karayolları", 
    "TCMB": "Türkiye Cumhuriyet Merkez Bankası", 
    "TCZB": "Türkiye Cumhuriyeti Ziraat Bankası", 
    "TDÇİ": "Türkiye Demir Çelik İşletmeleri", 
    "TDİ": "Türkiye Denizcilik İşletmeleri", 
    "TDK": "Türk Dil Kurumu", 
    "TEAE": "Tarımsal Ekonomi Araştırma Enstitüsü", 
    "TEAŞ": "Türkiye Elektrik Üretim Anonim Şirketi", 
    "TEDAŞ": "Türkiye Elektrik Dağıtım Anonim Şirketi", 
    "TEDGEM": "Teşkilatlanma ve Destekleme Genel Müdürlüğü", 
    "TEFE": "Toptan eşya fiyatları endeksi", 
    "TEİAŞ": "Türkiye Elektrik İletim Anonim Şirketi", 
    "TEK": "Türkiye Elektrik Kurumu", 
    "tek.": "Teknik", 
    "tel.": "Telefon", 
    "telg.": "Telgraf", 
    "TEM": "Avrupa Transit Kara Yolu",   
    "TEMSAN": "Türkiye Elektromekanik Sanayii", 
    "TETAŞ": "Türkiye Elektrik, Ticaret ve Taahhüt Anonim Şirketi",  
    "TFF": "Türkiye Futbol Federasyonu Başkanlığı",
    "TGC": "Türkiye Gazeteciler Cemiyeti", 
    "TGM": "Kültür ve Turizm Bakanlığı Tanıtma Genel Müdürlüğü", 
    "TGS": "Türkiye Gazeteciler Sendikası", 
    "Tğm.": "Teğmen", 
    "THA": "Türk Haberler Ajansı", 
    "THK": "Türk Hava Kurumu", 
    "THS": "Türk Havacılık Sanayii", 
    "THSGM": "Telif Hakları ve Sinema Genel Müdürlüğü", 
    "THY": "Türk Hava Yolları", 
    "TIR": "Uluslararası Kara Yolu Taşımacılığı",
    "tic.": "Ticaret", 
    "TİGEM": "Tarım İşletmeleri Genel Müdürlüğü", 
    "TİKA": "Türk İş Birliği ve Koordinasyon Ajansı Başkanlığı", 
    "TİM": "Türkiye İhracatçılar Merkezi", 
    "TİSK": "Türkiye İşverenler Sendikası Konfederasyonu",
    "tiy.": "Tiyatro", 
    "TKAE": "Türk Kültürünü Araştırma Enstitüsü", 
    "TKB": "Türkiye Kalkınma Bankası", 
    "TKF": "Toplu Konut Fonu", 
    "TKGM": "Tapu ve Kadastro Genel Müdürlüğü", 
    "TKİ": "Türkiye Kömür İşletmeleri", 
    "TL": "Türk lirası",
    "tlks.": "Teleks", 
    "tls.": "Telsiz", 
    "TM": "Türk malı", 
    "TMMOB": "Türk Mühendis ve Mimar Odaları Birliği", 
    "TMO": "Toprak Mahsulleri Ofisi", 
    "TMSF": "Tasarruf Mevduatı Sigorta Fonu",  
    "TMSK": "Türkiye Muhasebe Standartları Kurulu", 
    "TNCÜ": "Tunceli Üniversitesi", 
    "TOBB": "Türkiye Odalar ve Borsalar Birliği", 
    "TOBB ETÜ": "TOBB Ekonomi ve Teknoloji Üniversitesi", 
    "TODAİE": "Türkiye ve Orta Doğu Amme İdaresi Enstitüsü", 
    "TOEFL": "Yabancı dil olarak İngilizce testi", 
    "TOKİ": "Toplu Konut İdaresi Başkanlığı", 
    "Top.": "Topçu", 
    "TÖMER": "Türkçe Öğretim Merkezi", 
    "TP": "Türk Petrolleri", 
    "TPAO": "Türkiye Petrolleri Anonim Ortaklığı", 
    "TPE": "Türk Patent Enstitüsü", 
    "TR": "Türkiye", 
    "TRGM": "Tarım Reformu Genel Müdürlüğü",
    "TRKGM": "Tüketicinin ve Rekabetin Korunması Genel Müdürlüğü", 
    "TRS": "Toros Üniversitesi", 
    "TRT": "Türkiye Radyo ve Televizyon Kurumu",
    "TSE": "Türk Standartları Enstitüsü", 
    "TSİ": "Türkiye saati ile", 
    "TSK": "Türk Silahlı Kuvvetleri",
    "TSKB": "Türkiye Sınai Kalkınma Bankası", 
    "TT": "Türkiye Türkçesi", 
    "TTB": "Türk Tabipleri Birliği", 
    "TTK": "Türk Tarih Kurumu", 
    "TTKB": "Talim ve Terbiye Kurulu Başkanlığı", 
    "TTOK": "Türkiye Turing ve Otomobil Kurumu", 
    "TUBİM": "Türkiye Uyuşturucu ve Uyuşturucu Bağımlılığını İzleme Merkezi", 
    "Tug.": "Tugay", 
    "Tuğa.": "Tuğamiral", 
    "Tuğg.": "Tuğgeneral",
    "PDR": "Psikolojik danışmanlık ve rehberlik",
}

In [ ]:
def convert_abbrev_in_text(text):
    tokens = word_tokenize(text)
    tokens = [kisaltmalar[word] if word in kisaltmalar.keys() else word for word in tokens]
    text = ' '.join(tokens)
    return text

In [ ]:
df["Text Cleaned"] = df["text"].apply(convert_abbrev_in_text)

In [ ]:
df['Text Cleaned'] = [token.lower() for token in df['Text Cleaned']]

In [ ]:
df['Text Cleaned'] = df['Text Cleaned'].apply(lambda x: re.sub('[0-9]+', '', x))

In [ ]:
df['Text Cleaned'] = df['Text Cleaned'].apply(lambda x: x.translate(x.maketrans('', '', string.punctuation)))
df['Text Cleaned'] = df['Text Cleaned'].apply(lambda x: x.replace('"', '').replace("’", '').replace("'", '').replace("”", ''))

In [ ]:
df['Text Cleaned'] = df['Text Cleaned'].apply(lambda x: re.sub('\S*@\S*\s?', '', x))

In [ ]:
df['Text Cleaned'] = df['Text Cleaned'].apply(lambda x: emoji.replace_emoji(x))

In [ ]:
df['Text Cleaned'] = df['Text Cleaned'].apply(lambda x: re.sub('<.*?>', '', x))

In [ ]:
stop_words = [x.strip() for x in open('/kaggle/input/zemberekwords/stop-words.tr.txt','r', encoding="UTF8").read().split('\n')]

In [ ]:
df['Text Cleaned'] = df['Text Cleaned'].apply(lambda text: ' '.join([word for word in text.split() if word.lower() not in stop_words]))

In [ ]:
freq = pd.Series(' '.join(df['Text Cleaned']).split()).value_counts()
less_freq = list(freq[freq == 1].index)

In [ ]:
df['Text Cleaned'] = df['Text Cleaned'].apply(lambda x: " ".join(x for x in x.split() if x not in less_freq))

In [ ]:
df["Text Cleaned"] = df["Text Cleaned"].apply(lambda x: x if len(x) > 1 else np.nan)

In [ ]:
df = df.dropna().reset_index(drop=True)

In [ ]:
df.sample(5)

# EDA

In [ ]:
df["label"].value_counts().plot(kind="pie", autopct="%.1f%%")

In [ ]:
tags = Counter(df["label"]).keys()
tags_len = Counter(df["label"]).values()
tag_df = pd.DataFrame(zip(tags, tags_len), columns=["Class", "Count"])
tag_df.plot(x="Class", y="Count", kind="bar", legend=False, grid=False, figsize=(15, 5))
plt.title("Class / Count", fontsize=18)
plt.xlabel("Class", fontsize=15)
plt.ylabel("Count", fontsize=15)
plt.show()

In [ ]:
def print_wordcloud(label):
    tokens = ''
    for token in df[df["label"] == label]["Text Cleaned"]:
        tokens += token

    wordcloud = WordCloud(background_color="white", width=1200, height=800).generate(tokens)

    plt.figure(figsize=(12, 8))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.title(f"{label} - Word Cloud")
    plt.show()

In [ ]:
print_wordcloud("kızgın")

In [ ]:
print_wordcloud("korku")

In [ ]:
print_wordcloud("mutlu")

In [ ]:
print_wordcloud("surpriz")

In [ ]:
print_wordcloud("üzgün")

In [ ]:
def count_ngrams(corpus, ngram, n):
    vec = CountVectorizer(ngram_range=(ngram,ngram)).fit(corpus)
    bow = vec.transform(corpus).sum(axis=0)
    words_freq = sorted([(word, bow[0, idx]) for word, idx in vec.vocabulary_.items()], key=lambda x: x[1], reverse=True)[:n]
    return words_freq

In [ ]:
def plot_ngrams(ngram_df, ngram_name):
    plt.figure(figsize=(12, 6))
    plt.bar(data=ngram_df, x="Text", height="Count")
    plt.xticks(rotation=90)
    plt.xlabel(ngram_name)
    plt.ylabel("Count")
    plt.title(ngram_name)
    plt.show()

In [ ]:
unigrams = count_ngrams(df["Text Cleaned"], 1, 30)
top_unigram = pd.DataFrame(unigrams, columns=['Text', "Count"])
top_unigram.head()

In [ ]:
plot_ngrams(top_unigram, "Unigrams")

In [ ]:
bigrams = count_ngrams(df["Text Cleaned"], 2, 30)
top_bigram = pd.DataFrame(bigrams, columns=['Text', "Count"])
top_bigram.head()

In [ ]:
plot_ngrams(top_bigram, "Bigrams")

In [ ]:
trigrams = count_ngrams(df["Text Cleaned"], 3, 30)
top_trigram = pd.DataFrame(trigrams, columns=['Text', "Count"])
top_trigram.head()

In [ ]:
plot_ngrams(top_trigram, "Trigrams")

# Feature Scaling

In [ ]:
le = LabelEncoder().fit(df["label"])
le_nm = dict(zip(le.classes_, le.transform(le.classes_)))
df["labels"] = df["label"].apply(lambda x: le_nm[x])

In [ ]:
df = df.drop(['text', 'label'], axis=1)

# BERT

In [ ]:
tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-turkish-uncased")
model = BertForSequenceClassification.from_pretrained("dbmdz/bert-base-turkish-uncased", num_labels=5).to(device)

In [ ]:
MAX_LEN = 128
BATCH_SIZE = 2
EPOCHS = 4
LEARNING_RATE = 2e-5

In [ ]:
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, item):
        text = self.texts[item]
        label = self.labels[item]
        
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
        )
        
        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(df['Text Cleaned'], df['labels'], test_size=0.1, random_state=42)

In [ ]:
train_dataset = SentimentDataset(train_texts.to_numpy(), train_labels.to_numpy(), tokenizer, MAX_LEN)
val_dataset = SentimentDataset(val_texts.to_numpy(), val_labels.to_numpy(), tokenizer, MAX_LEN)

In [ ]:
train_data_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_data_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
total_steps = len(train_data_loader) * EPOCHS
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

# Train

In [ ]:
def train_epoch(model, data_loader, optimizer, scheduler, device):
    model = model.train()
    losses = []
    correct_predictions = 0

    for d in data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        labels = d["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        _, preds = torch.max(logits, dim=1)
        correct_predictions += torch.sum(preds == labels)
        losses.append(loss.item())

        loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    return correct_predictions.double() / len(data_loader.dataset), np.mean(losses)

In [ ]:
def eval_model(model, data_loader, device):
    model = model.eval()
    losses = []
    correct_predictions = 0

    with torch.no_grad():
        for d in data_loader:
            input_ids = d["input_ids"].to(device)
            attention_mask = d["attention_mask"].to(device)
            labels = d["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            _, preds = torch.max(logits, dim=1)
            correct_predictions += torch.sum(preds == labels)
            losses.append(loss.item())

    return correct_predictions.double() / len(data_loader.dataset), np.mean(losses)

In [ ]:
for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    print('-' * 10)

    train_acc, train_loss = train_epoch(model, train_data_loader, optimizer, scheduler, device)
    print(f'Train loss {train_loss} accuracy {train_acc}')

    val_acc, val_loss = eval_model(model, val_data_loader, device)
    print(f'Val   loss {val_loss} accuracy {val_acc}')

# Test

In [ ]:
y_pred = []
y_true = []

model = model.eval()

with torch.no_grad():
    for d in val_data_loader:
        input_ids = d["input_ids"].to(device)
        attention_mask = d["attention_mask"].to(device)
        labels = d["labels"].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        _, preds = torch.max(logits, dim=1)

        y_pred.extend(preds.cpu().numpy())
        y_true.extend(labels.cpu().numpy())

# Results

In [ ]:
model_precision_score = precision_score(y_true, y_pred, average='weighted')
model_f1_score = f1_score(y_true, y_pred, average='weighted')
model_recall_score = recall_score(y_true, y_pred, average='weighted')
model_accuracy_score = accuracy_score(y_true, y_pred)

print(f"Precision Score = {model_precision_score * 100:.2f}%")
print(f"F1 Score = {model_f1_score * 100:.2f}%")
print(f"Recall Score = {model_recall_score * 100:.2f}%")
print(f"Accuracy Score = {model_accuracy_score * 100:.2f}%")

In [ ]:
print(classification_report(y_true, y_pred))

In [ ]:
cm = confusion_matrix(y_true, y_pred)
fig, ax = plot_confusion_matrix(conf_mat=cm, show_absolute=True, show_normed=True, colorbar=True, class_names=le.classes_)
plt.show()